In [1]:
import numpy as np
import cv2
import json

In [2]:
import os


In [3]:
# Label map
#box_labels = ('b', 't', 'm', 'k', 'cr','cg','cb','cx','cw','cy','cn')
#label_map = {k: v  for v, k in enumerate(box_labels)}
#label_map['background'] = 0
#rev_label_map = {v: k for k, v in label_map.items()}  # Inverse mapping


Elan_od_singapore_rev_label_map={}
Elan_od_singapore_labels = ('b', 't', 'm', 'k', 'c','car_plate','motor_plate')
Elan_od_singapore_label_map = {k: v + 1 for v, k in enumerate(Elan_od_singapore_labels)}
Elan_od_singapore_label_map['background'] = 0
Elan_od_singapore_rev_label_map = {v: k for k, v in Elan_od_singapore_label_map.items()}  # Inverse mapping

# Color map for bounding boxes of detected objects from https://sashat.me/2017/01/11/list-of-20-simple-distinct-colors/
Elan_od_singapore_distinct_colors = ['#e6194b', '#3cb44b', '#ffe119', '#0082c8', '#f58231', '#008080','#e6194b','#ebfa46']
Elan_od_singapore_label_color_map = {k: Elan_od_singapore_distinct_colors[i] for i, k in enumerate(Elan_od_singapore_label_map.keys())}

In [4]:
def fun_json2ObjLoc(json_filepath):
    with open(json_filepath, 'r') as f:
        data = json.load(f) 
    bboxes=[]
    for shape in data['shapes']:
        if shape['shape_type']=='rectangle':
            label = shape['label'] 
            if label in ['c_r','c_g','c_b','c_x','c_w','c_y','c_n','cr','cg','cb','cx','cw','cy','cn','c_x.','cp']:
                label = 'c'
            if label in ['c4','c5','c6','c7','cr','cl','c_4','c_5','c_6','c_7','c_r','c_l']:
                label = 'car_plate'
            if label in ['m4','m6','m7','mr','ml','m_4','m_6','m_7','m_r','m_l']:
                label = 'motor_plate'
            if label in ['mz','cz','m_z','c_z','dr','sw','ud','cz.']:
                continue

            points = shape['points']
            box={}
            box['label'] = Elan_od_singapore_label_map[label]
            x_min = min(points[0][0], points[1][0])
            y_min = min(points[0][1], points[1][1])
            x_max = max(points[0][0], points[1][0])
            y_max = max(points[0][1], points[1][1])
#            box['points']=[points[0][0],points[0][1], points[1][0], points[1][1]]
            box['points']=[int(x_min),int(y_min),int(x_max), int(y_max)]
            bboxes.append(box)
    return bboxes

In [5]:
def create_data_lists(img_path,label_path,output_folder,split='train'):
    img_path = os.path.abspath(img_path)
    label_path = os.path.abspath(label_path)
    print(img_path,label_path)
    train_images = list()
    train_objects = list()
    label = list()
    n_objects = 0
    for dirPath, dirNames, fileNames in os.walk(img_path):
        if 'Segmentation' in dirPath:
            print(dirPath)
            for __ ,_ , fileName in os.walk(dirPath):
                for name in sorted(fileName):
                    if '.jpg' in dirPath+'/'+name:
                        train_images.append(dirPath+'/'+name)
                    elif '.json' in dirPath+'/'+name:
                        #print(dirPath+'/'+name)
                        label.append(dirPath+'/'+name)
                        objects = fun_json2ObjLoc(dirPath+'/'+name)
                        if len(objects) == 0:
                            print(dirPath+'/'+name)
                            continue
                        n_objects += len(objects)
                        train_objects.append(objects)
            for i in range(len(label)):
                if train_images[i][:-3] !=  label[i][:-4]:
                    print(train_images[i],label[i])
            print(len(train_objects),len(train_images))
            assert len(train_objects) == len(train_images)
    if split == 'train':
        with open(os.path.join(output_folder, 'TRAIN_images.json'), 'w') as j:
            json.dump(train_images, j)
        with open(os.path.join(output_folder, 'TRAIN_objects.json'), 'w') as j:
            json.dump(train_objects, j)
        with open(os.path.join(output_folder, 'label_map.json'), 'w') as j:
            json.dump(Elan_od_singapore_label_map, j)  # save label map too
    elif split == 'val':
        with open(os.path.join(output_folder, 'VALID_images.json'), 'w') as j:
            json.dump(train_images, j)
        with open(os.path.join(output_folder, 'VALID_objects.json'), 'w') as j:
            json.dump(train_objects, j)
        with open(os.path.join(output_folder, 'label_map.json'), 'w') as j:
            json.dump(Elan_od_singapore_label_map, j)  # save label map too
    '''
    print(img_path)
    print(label_path)
    for filename in sorted(os.listdir(img_path)):
        if '.jpg' in filename:
            img = os.path.join(img_path+'/'+filename)
            train_images.append(img)
    #print(train_images)  
    for filename in sorted(os.listdir(label_path)): 
        if '.json' in filename:
            objects = fun_json2ObjLoc(os.path.join(label_path+'/'+filename))
            if len(objects) == 0:
                continue
            n_objects += len(objects)
            train_objects.append(objects)
    #print(train_objects)
    assert len(train_objects) == len(train_images)
    #train_images.sort()
    #train_objects.sort()
    with open(os.path.join(output_folder, 'TRAIN_images.json'), 'w') as j:
        json.dump(train_images, j)
    with open(os.path.join(output_folder, 'TRAIN_objects.json'), 'w') as j:
        json.dump(train_objects, j)
    with open(os.path.join(output_folder, 'label_map.json'), 'w') as j:
        json.dump(label_map, j)  # save label map too
    '''         

In [6]:
create_data_lists("/media/cvlab04/PlateData/","/media/cvlab04/PlateData/","/media/cvlab04/PlateData",'train')

/media/cvlab04/PlateData /media/cvlab04/PlateData
/media/cvlab04/PlateData/OV_002-Part2_OV_002_0001-Segmentation
836 836
/media/cvlab04/PlateData/(Front)IPCamera_20200305095701_p001-Segmentation
952 952
/media/cvlab04/PlateData/(Front)IPCamera_20200305102504_p012-Segmentation
1070 1070
/media/cvlab04/PlateData/(Front)IPCamera_20200305102504_p008-Segmentation
1099 1099
/media/cvlab04/PlateData/(Front)IPCamera_20200305095701_p011-Segmentation
1201 1201
/media/cvlab04/PlateData/(Front)IPCamera_20200305104910_p008-Segmentation
1314 1314
/media/cvlab04/PlateData/(Front)IPCamera_20200305095701_p031-Segmentation
1415 1415
/media/cvlab04/PlateData/OV_001-1-Segmentation
2426 2426
/media/cvlab04/PlateData/(Front)IPCamera_20200305102504_p015-Segmentation
2434 2434
/media/cvlab04/PlateData/(Front)IPCamera_20200305104910_p011-Segmentation
2552 2552
/media/cvlab04/PlateData/(Front)IPCamera_20200305095701_p010-Segmentation
2670 2670
/media/cvlab04/PlateData/(Front)IPCamera_20200305095701_p026-Segment

In [7]:
create_data_lists("/media/cvlab04/PlateData_val/","/media/cvlab04/PlateData_val/","/media/cvlab04/PlateData_val",'val')

/media/cvlab04/PlateData_val /media/cvlab04/PlateData_val
/media/cvlab04/PlateData_val/(Front)IPCamera_20200305104910_p005-Segmentation
113 113
/media/cvlab04/PlateData_val/(Front)IPCamera_20200305104910_p003-Segmentation
229 229
/media/cvlab04/PlateData_val/(Front)IPCamera_20200305104910_p004-Segmentation
317 317
